In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import pandas as pd
import numpy as np
import os
import json

In [2]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
df_bbidding.columns.to_list()

['timestamp_utc',
 'Mean_SolarRadiation_dwd',
 'SolarDownwardRadiation_RW_dwd_Mean_30min',
 'SolarDownwardRadiation_RW_dwd_Mean_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_30min',
 'SolarDownwardRadiation_dwd_Mean_Lag_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_24h',
 'Panel_Efficiency_dwd_mean',
 'Panel_Efficiency_dwd_std',
 'Panel_Temperature_dwd_mean',
 'Panel_Temperature_dwd_std',
 'Std_Temperature_dwd',
 'Mean_Temperature_dwd',
 'cos_hour',
 'cos_day',
 'solar_mw_lag_48h',
 'capacity_mwp_lag_48h',
 'Target_Capacity_MWP%_lag_48h',
 'Target_Capacity_MWP%',
 'Solar_MWh_credit',
 'reference_time',
 'valid_time',
 'RelativeHumidity_dwd',
 'Temperature_dwd',
 'WindDirection_dwd',
 'WindDirection:100_dwd',
 'WindSpeed^3_dwd',
 'WindSpeed:100^3_dwd',
 'WindSpeed_dwd',
 'WindSpeed:100_dwd',
 'WindSpeed^3:100_dwd',
 'RelativeHumidity_ncep',
 'Temperature_ncep',
 'WindDirection_ncep',
 'WindDirection:100_ncep',
 'WindSpeed^3_ncep',
 'WindSpeed:100^3_ncep',
 'WindSpeed_ncep',
 'WindSpeed:100_nc

In [3]:
df_bbidding1 = df_bbidding[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_bbidding1.loc[:,"market_price_lag96h"] = df_bbidding1["market_price"].shift(192)
df_bbidding1.loc[:,"imbalance_price_lag96h"] = df_bbidding1["imbalance_price"].shift(192)
df_bbidding1.loc[:,"day_ahead_price_lag1week"] = df_bbidding1["day_ahead_price"].shift(336)
df_bbidding1.loc[:,"volume_lag96h"] = df_bbidding1["volume"].shift(192)
df_bbidding1.dropna(inplace=True)
df_bbidding1

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,5,6,7,8,9,imbalance_price,market_price_lag96h,imbalance_price_lag96h,day_ahead_price_lag1week,volume_lag96h
336,2021-03-04 08:00:00+00:00,68.55,73.77,1201.25,17,-0.500000,0.467359,106.202828,126.692556,188.827864,...,248.421344,278.463233,323.809394,373.852140,463.068992,89.95,7.54,8.00,32.38,755.80
337,2021-03-04 08:30:00+00:00,69.25,73.77,1105.35,18,-0.500000,0.467359,109.169687,146.438753,203.691171,...,237.873544,290.936786,336.975817,382.262617,516.284064,89.00,-9.71,7.00,32.38,900.35
338,2021-03-04 09:00:00+00:00,75.18,70.86,1245.05,19,-0.707107,0.467359,85.628752,156.903984,199.220095,...,238.966779,300.425701,345.177645,375.303556,488.195210,92.00,55.58,33.65,17.34,497.60
339,2021-03-04 09:30:00+00:00,73.19,70.86,1590.00,20,-0.707107,0.467359,89.618646,163.000319,205.983800,...,263.775945,307.808096,356.190088,404.786859,445.870108,92.00,57.95,62.70,17.34,554.50
340,2021-03-04 10:00:00+00:00,74.12,63.72,1415.30,21,-0.866025,0.467359,140.104118,259.247292,313.074122,...,350.917349,469.297198,491.961995,584.346472,624.029818,98.00,53.11,34.10,14.25,739.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42280,2023-08-25 23:30:00+00:00,83.91,84.41,1201.85,2,0.965926,-0.591261,28.044030,85.584636,147.450968,...,257.545157,312.000836,389.209655,461.471346,565.748439,66.01,91.88,126.00,50.21,1271.20
42281,2023-08-26 00:00:00+00:00,82.13,79.96,1066.95,3,1.000000,-0.577292,-2.130944,44.062739,80.500728,...,152.946134,213.340381,285.856261,352.111989,494.824570,66.01,90.64,90.64,37.82,1643.15
42282,2023-08-26 00:30:00+00:00,78.76,79.96,997.60,4,1.000000,-0.577292,-53.850807,-31.030380,1.674412,...,80.555516,138.624007,177.132223,250.514362,333.223882,66.01,95.15,115.99,37.82,1599.25
42283,2023-08-26 01:00:00+00:00,77.47,73.84,913.25,5,0.965926,-0.577292,-42.706652,-38.988480,19.495393,...,72.199724,128.661011,157.102188,214.652927,345.416021,66.01,91.30,125.00,31.66,1619.70


In [4]:
# Original dataset
X = df_bbidding1[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                  "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
y = df_bbidding1["day_ahead_price"].values  # Convert to numpy

# Step 1: Split into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Step 2: Further split the training set into train and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

# Step 3: Standardize the data (use only training data to fit the scaler)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [5]:
import torch
import torch.nn as nn

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        # Layer sizes from the best trial
        layer_sizes = [256, 448, 192, 96]
        dropout_rates = [0.12338360578207397, 0.2192742565593194, 0.15708417985889997, 0.253419888887539]

        # Define the layers
        self.fc1 = nn.Linear(input_dim, layer_sizes[0])
        self.fc2 = nn.Linear(layer_sizes[0], layer_sizes[1])
        self.fc3 = nn.Linear(layer_sizes[1], layer_sizes[2])
        self.fc4 = nn.Linear(layer_sizes[2], layer_sizes[3])
        self.fc5 = nn.Linear(layer_sizes[3], 1)  # Output layer

        # Dropouts
        self.dropout1 = nn.Dropout(dropout_rates[0])
        self.dropout2 = nn.Dropout(dropout_rates[1])
        self.dropout3 = nn.Dropout(dropout_rates[2])
        self.dropout4 = nn.Dropout(dropout_rates[3])

        # Activation function (Swish)
        self.swish = nn.SiLU()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))  # First layer with ReLU
        x = self.dropout1(x)         # First dropout
        x = self.swish(self.fc2(x))  # Second layer with Swish
        x = self.dropout2(x)         # Second dropout
        x = self.swish(self.fc3(x))  # Third layer with Swish
        x = self.dropout3(x)         # Third dropout
        x = self.swish(self.fc4(x))  # Fourth layer with Swish
        x = self.dropout4(x)         # Fourth dropout
        x = self.fc5(x)              # Output layer (no activation for raw outputs)
        return x



In [6]:
# y_train = y_train.values
# y_test = y_test.values

In [7]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Step 4: Create TensorDataset and DataLoader for training, validation, and testing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Step 5: Initialize the model, loss function, and optimizer
input_dim = X_train_tensor.shape[1]
model = MLP(input_dim)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [8]:
import torch.nn as nn

# Define MAE loss
mae_loss = nn.L1Loss()

# Training parameters
num_epochs = 500
patience = 15  # Number of epochs to wait for improvement
min_relative_improvement = 0.000001  # 0.1% improvement threshold
best_val_loss = float('inf')
early_stopping_counter = 0

# Lists to store metrics
train_losses = []
val_losses = []
test_losses = []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = mae_loss(outputs, y_batch)  # Use MAE loss
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    avg_train_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    # Validation phase
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = mae_loss(val_outputs, y_val_tensor).item()  # Use MAE loss
        val_losses.append(val_loss)
        
        test_outputs = model(X_test_tensor)
        test_loss = mae_loss(test_outputs, y_test_tensor).item()  # Use MAE loss
        test_losses.append(test_loss)
    
    # Early stopping and relative improvement check
    if val_loss < best_val_loss:
        relative_improvement = (best_val_loss - val_loss) / best_val_loss
        if relative_improvement < min_relative_improvement and epoch > 10:
            print(f"Stopping due to small relative improvement: {relative_improvement:.6f}")
            break
        
        best_val_loss = val_loss
        early_stopping_counter = 0
        # Save best model
        torch.save(model.state_dict(), 'MLP_day_ahead_price.pth')

    else:
        early_stopping_counter += 1
    
    # Early stopping check
    if early_stopping_counter >= patience:
        print(f"Early stopping triggered after {epoch + 1} epochs")
        break
    
    # Print progress
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {avg_train_loss:.4f}, "
              f"Val Loss: {val_loss:.4f}, "
              f"Test Loss: {test_loss:.4f}")

# Load best model after training
model.load_state_dict(torch.load('MLP_day_ahead_price.pth'))

# Final evaluation
model.eval()
with torch.no_grad():
    final_test_outputs = model(X_test_tensor)
    final_test_loss = mae_loss(final_test_outputs, y_test_tensor).item()  # Use MAE loss
print(f"Final Test Loss: {final_test_loss:.4f}")


Epoch [10/500], Train Loss: 40.8103, Val Loss: 54.5960, Test Loss: 83.3734
Epoch [20/500], Train Loss: 39.7884, Val Loss: 53.8032, Test Loss: 74.5097
Epoch [30/500], Train Loss: 38.9043, Val Loss: 52.4097, Test Loss: 67.4767
Epoch [40/500], Train Loss: 38.6188, Val Loss: 52.9476, Test Loss: 62.5263
Early stopping triggered after 46 epochs
Final Test Loss: 63.9335


C:\Users\paulh\AppData\Local\Temp\ipykernel_24076\1724466222.py:72: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('MLP_day_ahead_price.pth')

In [9]:
current_dir = os.getcwd()

path_df = os.path.abspath(os.path.join(current_dir, '..', 'basic_files'))
df_total_solar = pd.read_csv(os.path.join(path_df, 'solar_total_production.csv'))
df_total_solar.generation_mw = df_total_solar.generation_mw *0.5
df_total_wind = pd.read_csv(os.path.join(path_df, 'wind_total_production.csv'))
df_total_wind.generation_mw = df_total_wind.generation_mw *0.5 - df_total_wind.boa
df_imbalance_price = pd.read_csv(os.path.join(path_df, 'imbalance_price.csv'))
df_day_ahead_price = pd.read_csv(os.path.join(path_df, 'day_ahead_price.csv'))
df_market_price = pd.read_csv(os.path.join(path_df, 'market_index.csv'))

# Get the path to the 'logs' directory in the parent directory
path = os.path.abspath(os.path.join(current_dir, '..', 'logs'))
files = os.listdir(path)
txt_files = [file for file in files if file.endswith('.txt')]
data = []
for file in txt_files:
    with open(os.path.join(path, file), 'r') as f:
        try:
            json_data = json.load(f)
            data.append(json_data)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from file: {file}")
date_name = []
for i in range(len(data)):
    date_name.append(data[i]["prediction_date"])


In [10]:
# Extrahiere Daten
dataframe_list = []

for entry in data:
    prediction_date = entry['prediction_date']
    
    # Iteriere durch jedes 'submission' Element
    for submission in entry['solution']['submission']:
        timestamp = submission['timestamp']
        probabilistic_forecast = submission['probabilistic_forecast']
        
        # Extrahiere die Werte von 'probabilistic_forecast' und füge sie der Liste hinzu
        row = {
            'prediction_date': prediction_date,
            'timestamp': timestamp,
            '1': probabilistic_forecast.get('10', None),
            '2': probabilistic_forecast.get('20', None),
            '3': probabilistic_forecast.get('30', None),
            '4': probabilistic_forecast.get('40', None),
            '5': probabilistic_forecast.get('50', None),
            '6': probabilistic_forecast.get('60', None),
            '7': probabilistic_forecast.get('70', None),
            '8': probabilistic_forecast.get('80', None),
            '9': probabilistic_forecast.get('90', None)
        }
        dataframe_list.append(row)

# Erstelle DataFrame
df_api_new = pd.DataFrame(dataframe_list)
df_api_new = df_api_new.groupby("timestamp").last().reset_index()
df_api_new

,timestamp,prediction_date,1,2,3,4,5,6,7,8,9
0,2024-10-03T22:00:00+00:00,2024-10-04,1355,487,963,1544,1330,867,326,1402,1357
1,2024-10-03T22:30:00+00:00,2024-10-04,788,584,1148,1320,1007,565,923,797,487
2,2024-10-03T23:00:00+00:00,2024-10-04,571,748,1001,1460,740,1017,1533,597,991
3,2024-10-03T23:30:00+00:00,2024-10-04,1476,1194,1512,334,1054,1572,744,1497,463
4,2024-10-04T00:00:00+00:00,2024-10-04,1352,1390,912,662,1505,1093,814,487,1340
...,...,...,...,...,...,...,...,...,...,...,...
667,2024-10-21T19:30:00+00:00,2024-10-21,263,343,406,434,482,501,514,547,590
668,2024-10-21T20:00:00+00:00,2024-10-21,266,345,405,436,485,504,516,551,592
669,2024-10-21T20:30:00+00:00,2024-10-21,257,341,397,441,482,508,515,550,585
670,2024-10-21T21:00:00+00:00,2024-10-21,249,338,389,435,478,508,514,553,587


In [11]:
from datetime import datetime, timedelta
df_api_new = df_api_new.rename(columns={
    "timestamp": "datetime"
})
df_api_new["datetime"] = pd.to_datetime(df_api_new["datetime"])
min_date = df_api_new.datetime.min() - timedelta(minutes=30)
datetimes = pd.date_range(end=min_date, periods=336, freq='30min')
datetimes = pd.date_range(end=min_date, periods=336, freq='30min')
df_half_hourly = pd.DataFrame({"datetime": datetimes})
df_half_hourly["datetime"] = pd.to_datetime(df_half_hourly["datetime"])
df_api_new = pd.merge(df_half_hourly, df_api_new, left_on='datetime', right_on='datetime', how='outer')
df_api_new

,datetime,prediction_date,1,2,3,4,5,6,7,8,9
0,2024-09-26 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-09-26 22:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-09-26 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-09-26 23:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-09-27 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1003,2024-10-21 19:30:00+00:00,2024-10-21,263.0,343.0,406.0,434.0,482.0,501.0,514.0,547.0,590.0
1004,2024-10-21 20:00:00+00:00,2024-10-21,266.0,345.0,405.0,436.0,485.0,504.0,516.0,551.0,592.0
1005,2024-10-21 20:30:00+00:00,2024-10-21,257.0,341.0,397.0,441.0,482.0,508.0,515.0,550.0,585.0
1006,2024-10-21 21:00:00+00:00,2024-10-21,249.0,338.0,389.0,435.0,478.0,508.0,514.0,553.0,587.0


In [12]:
df_day_ahead_price.timestamp_utc = pd.to_datetime(df_day_ahead_price.timestamp_utc)
df_market_price.timestamp_utc = pd.to_datetime(df_market_price.timestamp_utc)
df_imbalance_price.timestamp_utc = pd.to_datetime(df_imbalance_price.timestamp_utc)
df_api_new_merged = pd.merge(df_api_new,df_day_ahead_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_market_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_imbalance_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged["day_ahead_price"] = df_api_new_merged["price_x"].rename("day_ahead_price")
df_api_new_merged["market_price"] = df_api_new_merged["price_y"].rename("market_price")
df_api_new_merged["settlement_period"] = df_api_new_merged["settlement_period_x"].rename("settlement_period")
df_api_new_merged["cos_hour"] = np.cos(2*np.pi*df_api_new_merged["datetime"].dt.hour/24)
df_api_new_merged["cos_day"] = np.cos(2*np.pi*df_api_new_merged["datetime"].dt.day/7)
df_api_new_merged

,datetime,prediction_date,1,2,3,4,5,6,7,8,...,price_y,volume,timestamp_utc,settlement_date,settlement_period,imbalance_price,day_ahead_price,market_price,cos_hour,cos_day
0,2024-09-26 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21.21,1485.30,2024-09-26 22:00:00+00:00,2024-09-26,47.0,68.929966,7.20,21.21,0.866025,-0.222521
1,2024-09-26 22:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.46,1364.20,2024-09-26 22:30:00+00:00,2024-09-26,48.0,-28.480000,7.20,3.46,0.866025,-0.222521
2,2024-09-26 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.15,1688.95,2024-09-26 23:00:00+00:00,2024-09-27,1.0,-19.020000,3.87,3.15,0.965926,-0.222521
3,2024-09-26 23:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.68,1661.10,2024-09-26 23:30:00+00:00,2024-09-27,2.0,-18.000000,3.87,-3.68,0.965926,-0.222521
4,2024-09-27 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.52,1824.60,2024-09-27 00:00:00+00:00,2024-09-27,3.0,56.450000,0.45,-0.52,1.000000,0.623490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,2024-10-21 19:30:00+00:00,2024-10-21,263.0,343.0,406.0,434.0,482.0,501.0,514.0,547.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.258819,1.000000
1010,2024-10-21 20:00:00+00:00,2024-10-21,266.0,345.0,405.0,436.0,485.0,504.0,516.0,551.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.500000,1.000000
1011,2024-10-21 20:30:00+00:00,2024-10-21,257.0,341.0,397.0,441.0,482.0,508.0,515.0,550.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.500000,1.000000
1012,2024-10-21 21:00:00+00:00,2024-10-21,249.0,338.0,389.0,435.0,478.0,508.0,514.0,553.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.707107,1.000000


In [13]:
df_api_new_merged1 = df_api_new_merged[["datetime","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_api_new_merged1.loc[:,"market_price_lag96h"] = df_api_new_merged1["market_price"].shift(192)
df_api_new_merged1.loc[:,"imbalance_price_lag96h"] = df_api_new_merged1["imbalance_price"].shift(192)
df_api_new_merged1.loc[:,"day_ahead_price_lag1week"] = df_api_new_merged1["day_ahead_price"].shift(336)
df_api_new_merged1.loc[:,"volume_lag96h"] = df_api_new_merged1["volume"].shift(192)
df_api_new_merged1.dropna(inplace=True)
df_api_new_merged1

,datetime,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,5,6,7,8,9,imbalance_price,market_price_lag96h,imbalance_price_lag96h,day_ahead_price_lag1week,volume_lag96h
336,2024-10-03 22:00:00+00:00,71.32,78.47,1322.45,47.0,0.866025,-0.900969,1355.0,487.0,963.0,...,1330.0,867.0,326.0,1402.0,1357.0,61.00,12.85,-1.4600,7.20,1457.25
337,2024-10-03 22:30:00+00:00,71.42,78.47,1035.65,48.0,0.866025,-0.900969,788.0,584.0,1148.0,...,1007.0,565.0,923.0,797.0,487.0,50.00,-3.95,-1.4600,7.20,1367.60
338,2024-10-03 23:00:00+00:00,66.94,71.36,1378.55,1.0,0.965926,-0.900969,571.0,748.0,1001.0,...,740.0,1017.0,1533.0,597.0,991.0,93.50,-2.74,5.0000,3.87,1473.20
339,2024-10-03 23:30:00+00:00,60.03,71.36,1539.90,2.0,0.965926,-0.900969,1476.0,1194.0,1512.0,...,1054.0,1572.0,744.0,1497.0,463.0,93.50,3.85,5.0000,3.87,1799.05
340,2024-10-04 00:00:00+00:00,65.21,72.80,1818.40,3.0,1.000000,-0.900969,1352.0,1390.0,912.0,...,1505.0,1093.0,814.0,487.0,1340.0,93.00,0.03,-2.4304,0.45,1689.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,2024-10-15 19:30:00+00:00,88.67,80.72,2071.15,42.0,0.258819,0.623490,850.0,921.0,980.0,...,1060.0,1081.0,1104.0,1123.0,1142.0,105.00,84.79,66.0000,89.55,2107.60
722,2024-10-15 20:00:00+00:00,83.31,64.10,1709.85,43.0,0.500000,0.623490,897.0,964.0,1010.0,...,1074.0,1097.0,1111.0,1129.0,1151.0,104.90,85.72,66.6000,87.15,2231.60
723,2024-10-15 20:30:00+00:00,69.98,64.10,1551.30,44.0,0.500000,0.623490,910.0,984.0,1040.0,...,1101.0,1120.0,1122.0,1139.0,1154.0,64.00,88.05,69.0000,87.15,1923.85
724,2024-10-15 21:00:00+00:00,69.82,28.87,1055.25,45.0,0.707107,0.623490,938.0,1034.0,1069.0,...,1124.0,1135.0,1139.0,1152.0,1164.0,33.77,85.14,85.1400,81.06,1929.50


In [14]:
X_test = df_api_new_merged1[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                    "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
X_test = scaler.transform(X_test)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

model.eval()
with torch.no_grad():
    final_test_outputs = model(X_test_tensor)
    final_test_outputs = final_test_outputs.numpy().flatten()

df_api_new_merged1["day_ahead_price_predictions"] = final_test_outputs
df_api_new_merged1

,datetime,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,6,7,8,9,imbalance_price,market_price_lag96h,imbalance_price_lag96h,day_ahead_price_lag1week,volume_lag96h,day_ahead_price_predictions
336,2024-10-03 22:00:00+00:00,71.32,78.47,1322.45,47.0,0.866025,-0.900969,1355.0,487.0,963.0,...,867.0,326.0,1402.0,1357.0,61.00,12.85,-1.4600,7.20,1457.25,79.252037
337,2024-10-03 22:30:00+00:00,71.42,78.47,1035.65,48.0,0.866025,-0.900969,788.0,584.0,1148.0,...,565.0,923.0,797.0,487.0,50.00,-3.95,-1.4600,7.20,1367.60,72.177742
338,2024-10-03 23:00:00+00:00,66.94,71.36,1378.55,1.0,0.965926,-0.900969,571.0,748.0,1001.0,...,1017.0,1533.0,597.0,991.0,93.50,-2.74,5.0000,3.87,1473.20,79.937096
339,2024-10-03 23:30:00+00:00,60.03,71.36,1539.90,2.0,0.965926,-0.900969,1476.0,1194.0,1512.0,...,1572.0,744.0,1497.0,463.0,93.50,3.85,5.0000,3.87,1799.05,87.852982
340,2024-10-04 00:00:00+00:00,65.21,72.80,1818.40,3.0,1.000000,-0.900969,1352.0,1390.0,912.0,...,1093.0,814.0,487.0,1340.0,93.00,0.03,-2.4304,0.45,1689.75,102.016685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,2024-10-15 19:30:00+00:00,88.67,80.72,2071.15,42.0,0.258819,0.623490,850.0,921.0,980.0,...,1081.0,1104.0,1123.0,1142.0,105.00,84.79,66.0000,89.55,2107.60,115.745155
722,2024-10-15 20:00:00+00:00,83.31,64.10,1709.85,43.0,0.500000,0.623490,897.0,964.0,1010.0,...,1097.0,1111.0,1129.0,1151.0,104.90,85.72,66.6000,87.15,2231.60,109.484283
723,2024-10-15 20:30:00+00:00,69.98,64.10,1551.30,44.0,0.500000,0.623490,910.0,984.0,1040.0,...,1120.0,1122.0,1139.0,1154.0,64.00,88.05,69.0000,87.15,1923.85,113.013878
724,2024-10-15 21:00:00+00:00,69.82,28.87,1055.25,45.0,0.707107,0.623490,938.0,1034.0,1069.0,...,1135.0,1139.0,1152.0,1164.0,33.77,85.14,85.1400,81.06,1929.50,105.303818


In [15]:
#plot imbalance price and predictions
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_api_new_merged1["datetime"], y=df_api_new_merged1["day_ahead_price"], mode='lines', name='day_ahead_price'))
fig.add_trace(go.Scatter(x=df_api_new_merged1["datetime"], y=df_api_new_merged1["day_ahead_price_predictions"], mode='lines', name='day_ahead_price Predictions'))
fig.update_layout(title='day_ahead_price and Predictions', xaxis_title='Timestamp', yaxis_title='day_ahead_price Price')
fig.show()

In [16]:
#MAE calculation for imbalance price predictions
mae = np.mean(np.abs(df_api_new_merged1["day_ahead_price"] - df_api_new_merged1["day_ahead_price_predictions"]))
print(f"MAE for day_ahead_price predictions: {mae:.4f}")

MAE for day_ahead_price predictions: 54.0816


In [17]:
df_api_new_merged1.drop(columns=["datetime"]).corr()["day_ahead_price"]

market_price                   0.194831
day_ahead_price                1.000000
volume                        -0.111749
settlement_period              0.236378
cos_hour                      -0.382180
cos_day                        0.155752
1                             -0.079614
2                             -0.085393
3                             -0.073003
4                             -0.080123
5                             -0.086917
6                             -0.074921
7                             -0.090322
8                             -0.071097
9                             -0.079846
imbalance_price                0.590410
market_price_lag96h            0.183979
imbalance_price_lag96h         0.055424
day_ahead_price_lag1week       0.064813
volume_lag96h                  0.327153
day_ahead_price_predictions    0.043989
Name: day_ahead_price, dtype: float64

In [18]:
df_api_new_merged1.columns

Index(['datetime', 'market_price', 'day_ahead_price', 'volume',
       'settlement_period', 'cos_hour', 'cos_day', '1', '2', '3', '4', '5',
       '6', '7', '8', '9', 'imbalance_price', 'market_price_lag96h',
       'imbalance_price_lag96h', 'day_ahead_price_lag1week', 'volume_lag96h',
       'day_ahead_price_predictions'],
      dtype='object')

In [19]:
df_day_ahead_demand = pd.read_csv('D:/Users/paulh\Desktop/Domäneprojekt2/Energy_production_price_prediction/day_ahead_demand_forecast.csv')
df_margin_forecast = pd.read_csv('D:/Users/paulh\Desktop/Domäneprojekt2/Energy_production_price_prediction/margin_forecast.csv')
df_margin_forecast.forecast_date = pd.to_datetime(df_margin_forecast.forecast_date)
df_day_ahead_demand.timestamp_utc = pd.to_datetime(df_day_ahead_demand.timestamp_utc)

In [20]:
df_api_new_merged2 = pd.merge(df_api_new_merged1,df_day_ahead_demand, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged2["date"] = df_api_new_merged2["datetime"].dt.date
df_api_new_merged2["date"] = pd.to_datetime(df_api_new_merged2["date"])
df_api_new_merged2 = pd.merge(df_api_new_merged2,df_margin_forecast, left_on='date', right_on='forecast_date', how='left')

In [21]:
df_api_new_merged2.dropna(inplace=True)
df_api_new_merged2 = df_api_new_merged2.groupby("timestamp_utc").last().reset_index()
df_api_new_merged2

,timestamp_utc,datetime,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,...,settlement_date,settlement_period_y,boundary,publish_time_utc_x,transmission_system_demand,national_demand,date,forecast_date,publish_time_utc_y,margin
0,2024-10-04 00:00:00+00:00,2024-10-04 00:00:00+00:00,65.21,72.80,1818.40,3.0,1.000000,-0.900969,1352.0,1390.0,...,2024-10-04,3,N,2024-10-03T20:45:00Z,22051.0,21551.0,2024-10-04,2024-10-04,2024-10-02T15:00:00Z,9842.0
1,2024-10-04 00:30:00+00:00,2024-10-04 00:30:00+00:00,64.79,72.80,1784.30,4.0,1.000000,-0.900969,1279.0,410.0,...,2024-10-04,4,N,2024-10-03T20:45:00Z,21886.0,21386.0,2024-10-04,2024-10-04,2024-10-02T15:00:00Z,9842.0
2,2024-10-04 01:00:00+00:00,2024-10-04 01:00:00+00:00,64.26,65.30,1647.20,5.0,0.965926,-0.900969,1223.0,378.0,...,2024-10-04,5,N,2024-10-04T00:46:00Z,21486.0,20986.0,2024-10-04,2024-10-04,2024-10-02T15:00:00Z,9842.0
3,2024-10-04 01:30:00+00:00,2024-10-04 01:30:00+00:00,65.61,65.30,1406.60,6.0,0.965926,-0.900969,848.0,1046.0,...,2024-10-04,6,N,2024-10-04T00:46:00Z,21471.0,20971.0,2024-10-04,2024-10-04,2024-10-02T15:00:00Z,9842.0
4,2024-10-04 02:00:00+00:00,2024-10-04 02:00:00+00:00,58.19,71.46,1149.45,7.0,0.866025,-0.900969,1469.0,543.0,...,2024-10-04,7,N,2024-10-04T00:46:00Z,20947.0,20447.0,2024-10-04,2024-10-04,2024-10-02T15:00:00Z,9842.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,2024-10-15 19:30:00+00:00,2024-10-15 19:30:00+00:00,88.67,80.72,2071.15,42.0,0.258819,0.623490,850.0,921.0,...,2024-10-15,42,N,2024-10-15T17:45:00Z,31502.0,30582.0,2024-10-15,2024-10-15,2024-10-13T21:00:00Z,17741.0
374,2024-10-15 20:00:00+00:00,2024-10-15 20:00:00+00:00,83.31,64.10,1709.85,43.0,0.500000,0.623490,897.0,964.0,...,2024-10-15,43,N,2024-10-15T17:45:00Z,29584.0,28878.0,2024-10-15,2024-10-15,2024-10-13T21:00:00Z,17741.0
375,2024-10-15 20:30:00+00:00,2024-10-15 20:30:00+00:00,69.98,64.10,1551.30,44.0,0.500000,0.623490,910.0,984.0,...,2024-10-15,44,N,2024-10-15T17:45:00Z,28555.0,27834.0,2024-10-15,2024-10-15,2024-10-13T21:00:00Z,17741.0
376,2024-10-15 21:00:00+00:00,2024-10-15 21:00:00+00:00,69.82,28.87,1055.25,45.0,0.707107,0.623490,938.0,1034.0,...,2024-10-15,45,N,2024-10-15T20:45:00Z,27186.0,26056.0,2024-10-15,2024-10-15,2024-10-13T21:00:00Z,17741.0


In [22]:
df_api_new_merged2.columns.to_list()

['timestamp_utc',
 'datetime',
 'market_price',
 'day_ahead_price',
 'volume',
 'settlement_period_x',
 'cos_hour',
 'cos_day',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'imbalance_price',
 'market_price_lag96h',
 'imbalance_price_lag96h',
 'day_ahead_price_lag1week',
 'volume_lag96h',
 'day_ahead_price_predictions',
 'settlement_date',
 'settlement_period_y',
 'boundary',
 'publish_time_utc_x',
 'transmission_system_demand',
 'national_demand',
 'date',
 'forecast_date',
 'publish_time_utc_y',
 'margin']

In [23]:
df_api_new_merged2.day_ahead_price_predictions

0      102.016685
1       76.143272
2       86.171936
3       93.570015
4       82.225060
          ...    
373    115.745155
374    109.484283
375    113.013878
376    105.303818
377    102.703911
Name: day_ahead_price_predictions, Length: 378, dtype: float32

In [24]:
df_api_new_merged2_train = df_api_new_merged2[:200]
df_api_new_merged2_test = df_api_new_merged2[200:]

In [25]:
import torch
import torch.nn as nn

class SimpleModel(nn.Module):
    def __init__(self, input_dim):
        super(SimpleModel, self).__init__()
        
        # Zwei Hidden Layers
        self.fc1 = nn.Linear(input_dim, 128)  # Erster Hidden Layer
        self.fc2 = nn.Linear(128, 64)         # Zweiter Hidden Layer
        
        # Ausgangsschicht
        self.fc3 = nn.Linear(64, 1)           # Ausgangsschicht
        
        # Dropout und Aktivierungsfunktion
        self.dropout = nn.Dropout(0.2)
        self.swish = nn.SiLU()

    def forward(self, x):
        # Durch die Hidden Layers
        x = self.swish(self.fc1(x))  # Erster Hidden Layer
        x = self.dropout(x)
        x = self.swish(self.fc2(x))  # Zweiter Hidden Layer
        x = self.dropout(x)
        
        # Ausgang
        x = self.fc3(x)  # Ausgangsschicht
        
        return x


In [26]:
cobined_X_train = df_api_new_merged2_train[["day_ahead_price_predictions","national_demand","transmission_system_demand","margin"]]
cobined_y_train = df_api_new_merged2_train["day_ahead_price"]

cobined_X_test = df_api_new_merged2_test[["day_ahead_price_predictions","national_demand","transmission_system_demand","margin"]]
cobined_y_test = df_api_new_merged2_test["day_ahead_price"]

cobined_X_train = scaler.fit_transform(cobined_X_train)
cobined_X_test = scaler.transform(cobined_X_test)

with open('scaler_MLP_extension_day_ahead.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [27]:
combined_X_train_tensor = torch.tensor(cobined_X_train, dtype=torch.float32)
combined_y_train_tensor = torch.tensor(cobined_y_train.values, dtype=torch.float32).view(-1, 1)
combined_X_test_tensor = torch.tensor(cobined_X_test, dtype=torch.float32)
combined_y_test_tensor = torch.tensor(cobined_y_test.values, dtype=torch.float32).view(-1, 1)

# Step 4: Create TensorDataset and DataLoader for training and testing
combined_train_dataset = TensorDataset(combined_X_train_tensor, combined_y_train_tensor)
combined_test_dataset = TensorDataset(combined_X_test_tensor, combined_y_test_tensor)

# DataLoader
combined_train_loader = DataLoader(dataset=combined_train_dataset, batch_size=16, shuffle=False)
combined_test_loader = DataLoader(dataset=combined_test_dataset, batch_size=16, shuffle=False)

# Step 5: Initialize the model, loss function, and optimizer
input_dim = combined_X_train_tensor.shape[1]
model = SimpleModel(input_dim)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Define MAE loss
mae_loss = nn.L1Loss()

# Training parameters
num_epochs = 100

In [28]:
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in combined_train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = mae_loss(outputs, y_batch)  # Use MAE loss
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    avg_train_loss = epoch_loss / len(combined_train_loader)
    train_losses.append(avg_train_loss)
    
    # Test phase
    model.eval()
    with torch.no_grad():
        test_outputs = model(combined_X_test_tensor)
        test_loss = mae_loss(test_outputs, combined_y_test_tensor).item()  # Use MAE loss
        test_losses.append(test_loss)
    
    # Print progress
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {avg_train_loss:.4f}, "
              f"Test Loss: {test_loss:.4f}")
        
torch.save(model.state_dict(), 'MLP_extension_day_ahead.pth')

# Final evaluation
model.eval()
with torch.no_grad():
    final_test_outputs = model(combined_X_test_tensor)
    final_test_loss = mae_loss(final_test_outputs, combined_y_test_tensor).item()  # Use MAE loss
print(f"Final Test Loss: {final_test_loss:.4f}")


Epoch [5/100], Train Loss: 83.1128, Test Loss: 83.5832
Epoch [10/100], Train Loss: 82.8784, Test Loss: 83.3475
Epoch [15/100], Train Loss: 82.5766, Test Loss: 83.0550
Epoch [20/100], Train Loss: 82.2045, Test Loss: 82.6779
Epoch [25/100], Train Loss: 81.6483, Test Loss: 82.1721
Epoch [30/100], Train Loss: 80.9934, Test Loss: 81.5095
Epoch [35/100], Train Loss: 80.1359, Test Loss: 80.6516
Epoch [40/100], Train Loss: 78.9347, Test Loss: 79.6033
Epoch [45/100], Train Loss: 77.6350, Test Loss: 78.3149
Epoch [50/100], Train Loss: 76.0237, Test Loss: 76.7793
Epoch [55/100], Train Loss: 73.9595, Test Loss: 74.9764
Epoch [60/100], Train Loss: 71.8505, Test Loss: 72.8828
Epoch [65/100], Train Loss: 69.1759, Test Loss: 70.5081
Epoch [70/100], Train Loss: 66.3480, Test Loss: 67.8856
Epoch [75/100], Train Loss: 63.1575, Test Loss: 65.0881
Epoch [80/100], Train Loss: 59.7300, Test Loss: 62.1691
Epoch [85/100], Train Loss: 55.8179, Test Loss: 59.1137
Epoch [90/100], Train Loss: 52.9559, Test Loss: 5

In [29]:
model.eval()
with torch.no_grad():
    final_test_outputs = model(combined_X_test_tensor)
    final_test_outputs = final_test_outputs.numpy().flatten()

df_api_new_merged2_test["day_ahead_price_predictions_2"] = final_test_outputs

C:\Users\paulh\AppData\Local\Temp\ipykernel_24076\421217961.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [30]:
df_api_new_merged2_test

,timestamp_utc,datetime,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,...,settlement_period_y,boundary,publish_time_utc_x,transmission_system_demand,national_demand,date,forecast_date,publish_time_utc_y,margin,day_ahead_price_predictions_2
200,2024-10-12 04:30:00+00:00,2024-10-12 04:30:00+00:00,57.89,85.61,1555.85,12.0,5.000000e-01,-0.222521,19.0,16.0,...,12,N,2024-10-12T02:46:00Z,20970.0,20051.0,2024-10-12,2024-10-12,2024-10-10T21:00:00Z,16509.0,66.457039
201,2024-10-12 05:00:00+00:00,2024-10-12 05:00:00+00:00,58.51,95.05,1808.75,13.0,2.588190e-01,-0.222521,18.0,16.0,...,13,N,2024-10-12T02:46:00Z,22492.0,21169.0,2024-10-12,2024-10-12,2024-10-10T21:00:00Z,16509.0,62.243298
202,2024-10-12 05:30:00+00:00,2024-10-12 05:30:00+00:00,62.21,95.05,2044.45,14.0,2.588190e-01,-0.222521,18.0,16.0,...,14,N,2024-10-12T02:46:00Z,23551.0,22233.0,2024-10-12,2024-10-12,2024-10-10T21:00:00Z,16509.0,58.656673
203,2024-10-12 06:00:00+00:00,2024-10-12 06:00:00+00:00,71.10,97.99,2242.90,15.0,6.123234e-17,-0.222521,21.0,19.0,...,15,N,2024-10-12T02:46:00Z,24440.0,23438.0,2024-10-12,2024-10-12,2024-10-10T21:00:00Z,16509.0,55.527187
204,2024-10-12 06:30:00+00:00,2024-10-12 06:30:00+00:00,79.52,97.99,2234.00,16.0,6.123234e-17,-0.222521,21.0,19.0,...,16,N,2024-10-12T06:16:00Z,25316.0,24410.0,2024-10-12,2024-10-12,2024-10-10T21:00:00Z,16509.0,53.006134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,2024-10-15 19:30:00+00:00,2024-10-15 19:30:00+00:00,88.67,80.72,2071.15,42.0,2.588190e-01,0.623490,850.0,921.0,...,42,N,2024-10-15T17:45:00Z,31502.0,30582.0,2024-10-15,2024-10-15,2024-10-13T21:00:00Z,17741.0,47.779892
374,2024-10-15 20:00:00+00:00,2024-10-15 20:00:00+00:00,83.31,64.10,1709.85,43.0,5.000000e-01,0.623490,897.0,964.0,...,43,N,2024-10-15T17:45:00Z,29584.0,28878.0,2024-10-15,2024-10-15,2024-10-13T21:00:00Z,17741.0,49.721088
375,2024-10-15 20:30:00+00:00,2024-10-15 20:30:00+00:00,69.98,64.10,1551.30,44.0,5.000000e-01,0.623490,910.0,984.0,...,44,N,2024-10-15T17:45:00Z,28555.0,27834.0,2024-10-15,2024-10-15,2024-10-13T21:00:00Z,17741.0,51.236435
376,2024-10-15 21:00:00+00:00,2024-10-15 21:00:00+00:00,69.82,28.87,1055.25,45.0,7.071068e-01,0.623490,938.0,1034.0,...,45,N,2024-10-15T20:45:00Z,27186.0,26056.0,2024-10-15,2024-10-15,2024-10-13T21:00:00Z,17741.0,54.522308


In [31]:
#plot imbalance price and predictions
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_api_new_merged2_test["timestamp_utc"], y=df_api_new_merged2_test["day_ahead_price"], mode='lines', name='day_ahead_price'))
fig.add_trace(go.Scatter(x=df_api_new_merged2_test["timestamp_utc"], y=df_api_new_merged2_test["day_ahead_price_predictions_2"], mode='lines', name='day_ahead_price Predictions'))

In [32]:
#MAE calculation for imbalance price predictions
mae = np.mean(np.abs(df_api_new_merged2_test["day_ahead_price"] - df_api_new_merged2_test["day_ahead_price_predictions_2"]))
mae

48.975235530981855

In [33]:
df_api_new_merged2_test.day_ahead_price.corr(df_api_new_merged2_test.day_ahead_price_predictions_2)

-0.23109469658746454

In [34]:
df_api_new_merged2_test.columns

Index(['timestamp_utc', 'datetime', 'market_price', 'day_ahead_price',
       'volume', 'settlement_period_x', 'cos_hour', 'cos_day', '1', '2', '3',
       '4', '5', '6', '7', '8', '9', 'imbalance_price', 'market_price_lag96h',
       'imbalance_price_lag96h', 'day_ahead_price_lag1week', 'volume_lag96h',
       'day_ahead_price_predictions', 'settlement_date', 'settlement_period_y',
       'boundary', 'publish_time_utc_x', 'transmission_system_demand',
       'national_demand', 'date', 'forecast_date', 'publish_time_utc_y',
       'margin', 'day_ahead_price_predictions_2'],
      dtype='object')

In [35]:
#correlation of day_ahead price with every other feature

df_api_new_merged2_test.drop(columns=["datetime","timestamp_utc","publish_time_utc_x","publish_time_utc_y","forecast_date","boundary","settlement_date"]).corr()["day_ahead_price"]

market_price                     0.083463
day_ahead_price                  1.000000
volume                          -0.298667
settlement_period_x              0.183181
cos_hour                        -0.431111
cos_day                          0.331204
1                               -0.185780
2                               -0.172937
3                               -0.168851
4                               -0.166870
5                               -0.166070
6                               -0.164493
7                               -0.162658
8                               -0.160187
9                               -0.154536
imbalance_price                  0.662266
market_price_lag96h              0.220853
imbalance_price_lag96h           0.055440
day_ahead_price_lag1week         0.058044
volume_lag96h                    0.344985
day_ahead_price_predictions      0.109252
settlement_period_y              0.183181
transmission_system_demand       0.606631
national_demand                  0

In [36]:
df_api_new_merged2_train.drop(columns=["datetime","timestamp_utc","publish_time_utc_x","publish_time_utc_y","forecast_date","boundary","settlement_date"]).corr()["day_ahead_price"]

market_price                   0.761854
day_ahead_price                1.000000
volume                         0.309640
settlement_period_x            0.305376
cos_hour                      -0.314224
cos_day                        0.076854
1                              0.050448
2                              0.028973
3                              0.077314
4                              0.041510
5                              0.033956
6                              0.063565
7                             -0.000216
8                              0.089793
9                              0.041531
imbalance_price                0.305652
market_price_lag96h            0.159313
imbalance_price_lag96h         0.028050
day_ahead_price_lag1week       0.099828
volume_lag96h                  0.300017
day_ahead_price_predictions    0.060623
settlement_period_y            0.305376
transmission_system_demand     0.543651
national_demand                0.573788
date                          -0.060165
